##### 版權 2024 Google LLC.


In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# REST API：調整入門指南


<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/tuning_quickstart_rest"><img src="https://ai.google.dev/static/site-assets/images/docs/notebook-site-button.png" height="32" width="32" />檢視 ai.google.dev</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />在 Google Colab 中執行</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/doggy8088/generative-ai-docs/blob/main/site/zh/tutorials/tuning_quickstart_rest.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />在 GitHub 上檢視來源</a>
  </td>
  <td>
    <a target="_blank" href="https://ai.google.dev/tutorials/tuning_quickstart_rest"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />下載筆記本</a>
  </td>
</table>


在這個筆記本中，你將瞭解如何使用 curl 指令或 Python 請求 API 來呼叫 Gemini API，以開始使用 Gemini API 調整服務。在此，你會瞭解如何調整 Gemini API 文字生成服務背後的文字模型。


**注意** ：目前僅支援 `gemini-1.0-pro-001` 模型進行微調。


## 設定


### 驗證


Gemini API 讓你能對你自己的資料進行模型的微調。由於這關係到你的資料以及你所進行微調的模型，因此需要比 API 金鑰更嚴格的存取控制。

在執行這個教學課程之前，你需要
[為你的專案設定 OAuth](oauth_quickstart.ipynb)。


在 Colab 中，設定的最簡單方法即複製你的 `client_secret.json` 檔案內容到 Colab 的「Secrets 管理員」中 (在左側面板的鑰匙圖示下方)，其密碼名稱為 `CLIENT_SECRET`。


此 gcloud 命令會將 `client_secret.json` 檔案轉換為可用于驗證服務的憑證。

> 重要事項：如果你在 Colab 上執行此操作，**請勿直接按一下印出的連結** 。這將失敗。請按照說明操作，並將印出的 `gcloud` 命令複製到你的本機機器並在當地執行，然後將本機機器的輸出貼回此處。


In [ ]:
try:
  from google.colab import userdata
  import pathlib
  pathlib.Path('client_secret.json').write_text(userdata.get('CLIENT_SECRET'))

  # Use `--no-browser` in colab
  !gcloud auth application-default login --no-browser --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'
except ImportError:
  !gcloud auth application-default login --client-id-file client_secret.json --scopes='https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/generative-language.tuning'

You are authorizing client libraries without access to a web browser. Please run the following command on a machine with a web browser and copy its output back here. Make sure the installed gcloud version is 372.0.0 or newer.

gcloud auth application-default login --remote-bootstrap="https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=87071151422-n1a3cb6c7fvkfg4gmhdtmn5ulol2l4be.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgenerative-language.tuning&state=QIyNibWSaTIsozjmvZEkVBo6EcoW0G&access_type=offline&code_challenge=76c1ZiGvKN8cvlYfj3BmbCwE4e7tvrlwaX3REUX25gY&code_challenge_method=S256&token_usage=remote"


Enter the output of the above command: https://localhost:8085/?state=QIyNibWSaTIsozjmvZEkVBo6EcoW0G&code=4/0AeaYSHBKrY911S466QjKQIFODoOPXlO1mWyTYYdrbELIDV6Hw2DKRAyro62BugroSvIWsA&scope=https://www.googleapis.com/auth/cloud-platform%20https://www.googleapis.com/auth/generativ

## 使用 CURL 呼叫 REST API


本節提供範例的 curl 陳述式以呼叫 REST API。你將學習如何建立調整工作、檢查其狀態並且一完成就發出推論呼叫。


### 設定變數


設定變數供重複值在後續其他 REST API 呼叫時使用。程式碼使用 Python `os` 函式庫設定環境變數，可在所有程式碼單元存取。

這特定於 Colab notebook 環境。下一個程式碼單元中的程式碼等於在 bash 終端機中執行下列命令。

```bash
export access_token=$(gcloud auth application-default print-access-token)
export project_id=my-project-id
export base_url=https://generativelanguage.googleapis.com
```


In [ ]:
import os

access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

os.environ['access_token'] = access_token
os.environ['project_id'] = "[Enter your project-id here]"
os.environ['base_url'] = "https://generativelanguage.googleapis.com"


### 列出已調整模型

透過列出目前可用的已調整模型來驗證你的驗證設定。


In [ ]:
%%bash

curl -X GET ${base_url}/v1beta/tunedModels \
    -H "Content-Type: application/json" \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}"


### 建立調整過的模型

如要建立調整過的模型，你需要在 `training_data` 欄位傳遞資料集給模型。

在此例中，你將調整模型以產生序列中的下一個數字。例如，輸入為  "1"  時，模型應輸出  "2"。輸入為  "一百"  時，輸出應為  "一百零一"。


In [ ]:
%%bash

curl -X POST $base_url/v1beta/tunedModels \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '
      {
        "display_name": "number generator model",
        "base_model": "models/gemini-1.0-pro-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 2,
            "learning_rate": 0.001,
            "epoch_count":5,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    "text_input": "1",
                    "output": "2",
                },{
                    "text_input": "3",
                    "output": "4",
                },{
                    "text_input": "-3",
                    "output": "-2",
                },{
                    "text_input": "twenty two",
                    "output": "twenty three",
                },{
                    "text_input": "two hundred",
                    "output": "two hundred one",
                },{
                    "text_input": "ninety nine",
                    "output": "one hundred",
                },{
                    "text_input": "8",
                    "output": "9",
                },{
                    "text_input": "-98",
                    "output": "-97",
                },{
                    "text_input": "1,000",
                    "output": "1,001",
                },{
                    "text_input": "10,100,000",
                    "output": "10,100,001",
                },{
                    "text_input": "thirteen",
                    "output": "fourteen",
                },{
                    "text_input": "eighty",
                    "output": "eighty one",
                },{
                    "text_input": "one",
                    "output": "two",
                },{
                    "text_input": "three",
                    "output": "four",
                },{
                    "text_input": "seven",
                    "output": "eight",
                }
              ]
            }
          }
        }
      }' | tee tunemodel.json


{
  "name": "tunedModels/number-generator-model-dzlmi0gswwqb/operations/bvl8dymw0fhw",
  "metadata": {
    "@type": "type.googleapis.com/google.ai.generativelanguage.v1beta.CreateTunedModelMetadata",
    "totalSteps": 38,
    "tunedModel": "tunedModels/number-generator-model-dzlmi0gswwqb"
  }
}


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2280    0   296  100  1984    611   4098 --:--:-- --:--:-- --:--:--  4720


### 取得已調整的模型狀態


在訓練期間，模型狀態設定為 `CREATING`，一旦完成將變更為 `ACTIVE`。

以下是少量的 python 程式碼，用於從回應 JSON 中分析產生的模型名稱。如果你在終端機執行這項工作，你可以嘗試使用 bash JSON parser 來分析回應。


In [ ]:
import json

first_page = json.load(open('tunemodel.json'))
os.environ['modelname'] = first_page['metadata']['tunedModel']

print(os.environ['modelname'])


tunedModels/number-generator-model-dzlmi0gswwqb


使用模型名稱執行另一個 `GET` 要求以取得模型中繼資料，其中包含狀態欄位。


In [ ]:
%%bash

curl -X GET ${base_url}/v1beta/${modelname} \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" | grep state


  "state": "ACTIVE",


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  5921    0  5921    0     0  13164      0 --:--:-- --:--:-- --:--:-- 13157


### 執行推論

在調整工作完成後，你可以使用文字服務產生文字。請嘗試輸入一個羅馬數字，例如 63 (LXIII)


In [ ]:
%%bash

curl -X POST $base_url/v1beta/$modelname:generateContent \
    -H 'Content-Type: application/json' \
    -H "Authorization: Bearer ${access_token}" \
    -H "x-goog-user-project: ${project_id}" \
    -d '{
        "contents": [{
        "parts": [{
          "text": "LXIII"
          }]
        }]
        }' 2> /dev/null

{
  "candidates": [
    {
      "content": {
        "parts": [
          {
            "text": "LXIV"
          }
        ],
        "role": "model"
      },
      "finishReason": "STOP",
      "index": 0,
      "safetyRatings": [
        {
          "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HATE_SPEECH",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_HARASSMENT",
          "probability": "NEGLIGIBLE"
        },
        {
          "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
          "probability": "NEGLIGIBLE"
        }
      ]
    }
  ],
  "promptFeedback": {
    "safetyRatings": [
      {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "probability": "NEGLIGIBLE"
      },
      {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "probability": "NEGLIGIBLE"
      },
      {
        "category": "HARM_CATEGORY_H

模型的輸出可能正確或不正確。如果調校過的模型未達到你的要求標準，你可以嘗試新增更多高品質範例、調整超參數或為你的範例新增前導文字。你甚至可以根據你建立的第一個範例建立另一個調校過後的模型。

請參閱 [調校指南](../guide/model_tuning_guidance) 以取得更多提高效能的指南。


## 使用 Python requests 呼叫 REST API

你可以使用允許你傳送 http 要求的任何函式庫呼叫 rest API。
下一組範例使用 Python requests 函式庫，並展示更進階功能。


### 設定變數


In [ ]:
access_token = !gcloud auth application-default print-access-token
access_token = '\n'.join(access_token)

project = '[Enter your project-id here]'
base_url = "https://generativelanguage.googleapis.com"


匯入`requests`函式庫。


In [ ]:
import requests
import json

### 列出已調整模型

透過列出目前可用的已調整模型來驗證你的驗證設定。


In [ ]:
headers={
  'Authorization': 'Bearer ' + access_token,
  'Content-Type': 'application/json',
  'x-goog-user-project': project
}

result = requests.get(
  url=f'{base_url}/v1beta/tunedModels',
  headers = headers,
)

In [ ]:
result.json()

### 建立調整過的模型


與 Curl 範例相同，你透過 `training_data` 欄位傳入資料集。


In [ ]:
operation = requests.post(
    url = f'{base_url}/v1beta/tunedModels',
    headers=headers,
    json= {
        "display_name": "number generator",
        "base_model": "models/gemini-1.0-pro-001",
        "tuning_task": {
          "hyperparameters": {
            "batch_size": 4,
            "learning_rate": 0.001,
            "epoch_count":5,
          },
          "training_data": {
            "examples": {
              "examples": [
                {
                    'text_input': '1',
                    'output': '2',
                },{
                    'text_input': '3',
                    'output': '4',
                },{
                    'text_input': '-3',
                    'output': '-2',
                },{
                    'text_input': 'twenty two',
                    'output': 'twenty three',
                },{
                    'text_input': 'two hundred',
                    'output': 'two hundred one',
                },{
                    'text_input': 'ninety nine',
                    'output': 'one hundred',
                },{
                    'text_input': '8',
                    'output': '9',
                },{
                    'text_input': '-98',
                    'output': '-97',
                },{
                    'text_input': '1,000',
                    'output': '1,001',
                },{
                    'text_input': '10,100,000',
                    'output': '10,100,001',
                },{
                    'text_input': 'thirteen',
                    'output': 'fourteen',
                },{
                    'text_input': 'eighty',
                    'output': 'eighty one',
                },{
                    'text_input': 'one',
                    'output': 'two',
                },{
                    'text_input': 'three',
                    'output': 'four',
                },{
                    'text_input': 'seven',
                    'output': 'eight',
                }
              ]
            }
          }
        }
      }
)

In [ ]:
operation

<Response [200]>

In [ ]:
operation.json()

{'name': 'tunedModels/number-generator-wl1qr34x2py/operations/41vni3zk0a47',
 'metadata': {'@type': 'type.googleapis.com/google.ai.generativelanguage.v1beta.CreateTunedModelMetadata',
  'totalSteps': 19,
  'tunedModel': 'tunedModels/number-generator-wl1qr34x2py'}}

設定一個變數，其名稱為已調整模型，用於其餘呼叫。


In [ ]:
name=operation.json()["metadata"]["tunedModel"]
name


'tunedModels/number-generator-wl1qr34x2py'

### 取得已調整的模型狀態


你可以透過檢查狀態欄位，來了解調整工作進度。`CREATING` 表示調整工作仍在進行中，而 `ACTIVE` 表示訓練完成，而且經過調整的模型已準備好使用。


In [ ]:
tuned_model = requests.get(
    url = f'{base_url}/v1beta/{name}',
    headers=headers,
)

In [ ]:
tuned_model.json()

以下程式碼每 5 秒檢查一次狀態欄位，直到它不再處於 `CREATING` 狀態。


In [ ]:
import time
import pprint

op_json = operation.json()
response = op_json.get('response')
error = op_json.get('error')

while response is None and error is None:
    time.sleep(5)

    operation = requests.get(
        url = f'{base_url}/v1/{op_json["name"]}',
        headers=headers,
    )

    op_json = operation.json()
    response = op_json.get('response')
    error = op_json.get('error')

    percent = op_json['metadata'].get('completedPercent')
    if percent is not None:
      print(f"{percent:.2f}% - {op_json['metadata']['snapshots'][-1]}")
      print()

if error is not None:
    raise Exception(error)

100.00% - {'step': 19, 'epoch': 5, 'meanLoss': 1.402067, 'computeTime': '2024-03-14T15:11:23.766989274Z'}



### 執行推斷

一旦調校作業完成，使用它來產生文字的方式與使用基礎文字模型的方式相同。嘗試輸入一個日文數字，例如，6 (六)。


In [ ]:
import time

m = requests.post(
    url = f'{base_url}/v1beta/{name}:generateContent',
    headers=headers,
    json= {
         "contents": [{
             "parts": [{
                 "text": "六"
             }]
          }]
    })

In [ ]:
import pprint
pprint.pprint(m.json())

{'candidates': [{'content': {'parts': [{'text': '七'}], 'role': 'model'},
                 'finishReason': 'STOP',
                 'index': 0,
                 'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
                                    'probability': 'NEGLIGIBLE'},
                                   {'category': 'HARM_CATEGORY_HATE_SPEECH',
                                    'probability': 'NEGLIGIBLE'},
                                   {'category': 'HARM_CATEGORY_HARASSMENT',
                                    'probability': 'LOW'},
                                   {'category': 'HARM_CATEGORY_DANGEROUS_CONTENT',
                                    'probability': 'NEGLIGIBLE'}]}],
 'promptFeedback': {'safetyRatings': [{'category': 'HARM_CATEGORY_SEXUALLY_EXPLICIT',
                                       'probability': 'NEGLIGIBLE'},
                                      {'category': 'HARM_CATEGORY_HATE_SPEECH',
                                       'pro

你模型的輸出是否正確尚不明確。如果調整後的模型未達到你的所需標準，你可以嘗試添加更多高品質範例、調整超參數或為範例添加前置頁。


## 結論

即使訓練資料沒有包含任何羅馬或日語數字的參考，模型在微調後能夠良好地進行泛化。這麼一來，你就可以微調模型來滿足你的使用案例。


##後續步驟

如要瞭解如何利用 Gemini API 的 Python SDK 使用調整服務，請前往[使用 Python 的調整快速入門](https://ai.google.dev/tutorials/tuning_quickstart_python)。如要瞭解如何使用 Gemini API 的其他服務，請前往[Python 快速入門](https://ai.google.dev/tutorials/python_quickstart)。
